In [2]:
from lightweight import lightweight

lightweight("Write a poem about dogs")

Loading model facebook/opt-125m on mps
Response: , or just about anything.
This is the best one, if you are in the mood for a little bit of a poem.


', or just about anything.\nThis is the best one, if you are in the mood for a little bit of a poem.'